In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
conf = pyspark.SparkConf() \
            .setAppName("syl") \
            .setMaster("spark://master:7077") \
            .set("spark.blockManager.port", "10025") \
            .set("spark.driver.blockManager.port", "10026") \
            .set("spark.driver.port", "10027") \
            .set("spark.cores.max", "2")
spark = SparkSession.builder.config(conf=conf).getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [8]:
spark

In [6]:
#spark.stop()

In [29]:
from pyarrow import fs
import pyarrow as pa
import subprocess
import os

classpath = subprocess.Popen(["/opt/hadoop/bin/hdfs", "classpath", "--glob"], stdout=subprocess.PIPE).communicate()[0]
os.environ["CLASSPATH"] = classpath.decode("utf-8")
hdfs = fs.HadoopFileSystem(host='192.168.0.160', port=8020, user='hadoop')

In [30]:
import datetime

today_news = datetime.datetime.now().strftime("%Y%m%d_mbc")

In [44]:
spark_df = spark.read.csv(f"hdfs:///P3T5/{today_news}.csv", 
               header=True, sep='|')

In [45]:
spark_df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Contents: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Organizer: string (nullable = true)



In [ ]:
spark_df.limit(5).show()